In [71]:
import numpy as np

class CNFConverter:
    def __init__(self, n, sentences, data):
        self.n = n
        self.sentences = sentences
        self.data = data
        self.memo = [[-1 for _ in range(n)] for _ in range(n)]

    def find_best_score_tree(self, start, end):
        if start == end:
            return self.data[start][end][0]

        scores = []
        for i in range(start, end):
            l = self.find_best_score_tree(start, i)
            r = self.find_best_score_tree(i + 1, end)
            scores.append(l + r)

        max_idx = np.argmax(scores)
        max_score = scores[max_idx]
        max_idx += start
        self.memo[start][end] = max_idx

        return max_score + self.data[start][end][0]

    def create_cnf(self, start, end):
        if start == end:
          return {
            'value': self.data[start][end][1],
            'children': [{'value': self.sentences[start], 'children': []}]
        }
        else:
          i = self.memo[start][end]
          return {
            'value': self.data[start][end][1],
            'children': [self.create_cnf(start, i), self.create_cnf(i + 1, end)]
            }

    def process_double_colon(self,tree):
        if '::' in tree['value']:
          value = tree['value'].split('::')
          children = tree['children']

          node = tree

          for i in range(len(value) - 1):
              node['value'] = value[i]
              node['children'] = [{}]
              node = node['children'][0]

          node['value'] = value[-1]
          node['children'] = children

    def truncate_parent(self, tree):
        if len(tree['children']) == 0:
            return

        for child_node in tree['children']:
            self.cnf2cfg(child_node)

        children_truncated = []
        for child_node in tree['children']:
            if '|<>' not in child_node['value']:
                children_truncated.append(child_node)
            else:
                children_truncated.extend(child_node['children'])

        tree['children'] = children_truncated

    def cnf2cfg(self, tree):
        self.process_double_colon(tree)
        self.truncate_parent(tree)

    def get_results(self, tree):
        res = '(' + tree['value']

        for child in tree['children']:
            res += self.get_results(child)

        res += ')'

        return res

In [72]:
def read_input(file_path):
    with open(file_path, 'r') as file:
      n = int(file.readline().strip())
      sentence = file.readline().strip().split()
      data = [[None for _ in range(n)] for _ in range(n)]

      for _ in range(int(n * (n + 1) / 2)):
          start, end, score, tag = file.readline().strip().split()
          start, end, score = int(start), int(end) - 1, float(score)
          data[start][end] = [score, tag]
      return n, sentence, data

#Testcase 1

In [73]:
n, sentence, data = read_input("/content/input1.txt")
cnf_converter = CNFConverter(n, sentence, data)
cnf_converter.find_best_score_tree(0, n - 1)
tree = cnf_converter.create_cnf(0, n - 1)
cnf_converter.cnf2cfg(tree)
print(cnf_converter.get_results(tree))

(S(NP-TMP(Hôm_nay))(NP-SUB(tôi))(VP(đi)(học))(.))


#Testcase2

In [74]:
n, sentence, data = read_input("/content/input2.txt")
cnf_converter = CNFConverter(n, sentence, data)
cnf_converter.find_best_score_tree(0, n - 1)
tree = cnf_converter.create_cnf(0, n - 1)
cnf_converter.cnf2cfg(tree)
print(cnf_converter.get_results(tree))

(S(NP(He))(VP(enjoys)(S(VP(playing)(NP(soccer)))))(.))
